In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/mm22d016/TransPolymer')
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torchmetrics
from rdkit import Chem
from torchmetrics import R2Score
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from copy import deepcopy

/home/mm22d016/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = pd.read_csv('../data/PE_II_aug.csv')
data

,SMILES descriptor 1,SMILES descriptor 2,Amion SMILES,log Li:functional group,Comonomer percentage,approxMW(kDa),approxTg,chain_architecture,logCond60
0,*COC(=O)OC*,*CCO*,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,0.07,95.20,37.0,-3,S_1,-5.720000
1,*COC(=O)OC*,*CCO*,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,0.20,95.20,37.0,-20,S_1,-5.190000
2,*COC(=O)OC*,*CCO*,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,0.45,95.20,37.0,-33,S_1,-4.810000
3,*COC(=O)OC*,*CCO*,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,1.20,95.20,37.0,-62,S_1,-4.250000
4,*COC(=O)OC*,*CCO*,O=S(=O)([N-]S(=O)(=O)C(F)(F)C(F)(F)F)C(F)(F)C(...,0.06,95.20,37.0,-14,S_1,-5.560000
...,...,...,...,...,...,...,...,...,...
266,*COC(*)COCCCC,*OC(*)=O,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,0.05,50.00,190.0,-20,S_2,-5.836820
267,*COC(*)COCCOC,*OC(*)=O,O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F,0.03,50.00,17.0,-55,S_2,-4.528742
268,*CCO*,*COC(*)COC,O=S(=O)([O-])C(F)(F)F,0.06,78.30,1000.0,-68,S_2,-4.771455
269,*CCO*,*COC(*)COCCOCC,O=S(=O)([O-])C(F)(F)F,0.06,82.94,1000.0,-53,S_2,-4.613009


In [15]:
unique_AmSM = np.unique(data['Amion SMILES'].values)
list(unique_AmSM)


['COCCC[N-]S(=O)(=O)C(F)(F)F',
 'F[B-](F)(F)F',
 'F[P-](F)(F)(F)(F)F',
 'N#C[S-]',
 'O=S(=O)(F)[N-]S(=O)(=O)F',
 'O=S(=O)([N-]S(=O)(=O)C(F)(F)C(F)(F)F)C(F)(F)C(F)(F)F',
 'O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F',
 'O=S(=O)([O-])C(F)(F)F',
 '[O-][Cl+3]([O-])([O-])[O-]']